In [11]:
"""
This training script can be run both on a single gpu in debug mode,
and also in a larger training run with distributed data parallel (ddp).

To run on a single GPU, example:
$ python train.py --batch_size=32 --compile=False

To run with DDP on 4 gpus on 1 node, example:
$ torchrun --standalone --nproc_per_node=4 train.py

To run with DDP on 4 gpus across 2 nodes, example:
- Run on the first (master) node with example IP 123.456.123.456:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=0 --master_addr=123.456.123.456 --master_port=1234 train.py
- Run on the worker node:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=1 --master_addr=123.456.123.456 --master_port=1234 train.py
(If your cluster does not have Infiniband interconnect prepend NCCL_IB_DISABLE=1)
"""

import os
import time
import math
import pickle
import json
from contextlib import nullcontext

import pandas as pd

import numpy as np
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

from model import GPTConfig, GPT

# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O
out_dir = 'out-stock'
eval_interval = 100
log_interval = 1
eval_iters = 2
eval_only = False # if True, script exits right after the first eval
always_save_checkpoint = False # if True, always save a checkpoint after each eval
init_from = 'scratch' # 'scratch' or 'resume' or 'gpt2*'
# wandb logging
wandb_log = False # disabled by default
wandb_project = 'stock'
wandb_run_name = 'mini-gpt' # 'run' + str(time.time())
# data
dataset = 'stock'
gradient_accumulation_steps = 1 # used to simulate larger batch sizes
batch_size = 12 # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 5
# model
n_layer = 8
n_head = 8
n_embd = 512 # 参数估计：params= layer*(12*emb**2), 数据量估计dataSize= 10 *params
# dropout = 0.998728434 # for pretraining 0 is good, for finetuning try 0.1+
dropout = 0
# dropout = 1-dataSize/10/params
bias = False # do we use bias inside LayerNorm and Linear layers?
meta_vocab_size = n_embd // 2

# adamw optimizer
learning_rate = 1e-3 # max learning rate
max_iters = 1000 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.99
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 100 # how many steps to warm up for
lr_decay_iters = 2000 # should be ~= max_iters per Chinchilla
min_lr = 1e-4 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla

# system
device = 'cpu' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
# -----------------------------------------------------------------------------
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
config = {k: globals()[k] for k in config_keys} # will be useful for logging
# -----------------------------------------------------------------------------

master_process = True
seed_offset = 0


if master_process:
    os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)


In [12]:
np.random.seed(42)

datadir = os.path.join('data', dataset)

# meta数据
meta = {}
with open(os.path.join(datadir, 'meta.pkl'), 'r') as f:
    meta = json.load(f)
    meta_vocab_size = meta['vocab_size']
    meta_vocab_size = 4096
def decode(id):
    return meta['itos'][str(id)]
def encode(s):
    return [meta['stoi'][c] for c in s]

pd_train_data = pd.read_csv(os.path.join(datadir, 'train.csv')).iloc[1:,:meta_vocab_size+1]
pd_val_data = pd.read_csv(os.path.join(datadir, 'val.csv')).iloc[:,:meta_vocab_size+1]
pd_val_data



,trade_date,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,...,603320.SH,603321.SH,603322.SH,603323.SH,603324.SH,603325.SH,603326.SH,603327.SH,603328.SH,603329.SH
0,20240206,1.0320,1.0359,0.9051,0.9506,1.0420,0.9950,1.0412,1.1000,0.9610,...,0.9091,0.9156,1.0424,1.0339,1.0262,1.0645,0.9214,1.0366,1.0486,0.9616
1,20240207,0.9928,1.0011,0.9859,0.9481,1.0161,0.9799,0.9901,1.0991,0.9036,...,0.9000,0.9136,0.9500,1.0211,0.9627,0.9671,0.9197,0.9729,1.0232,1.0999
2,20240208,1.0062,1.0567,1.0559,0.9452,1.0714,1.0308,1.0350,0.9942,1.0955,...,1.1000,1.1008,1.0954,0.9954,1.0999,1.0887,1.0727,1.0967,1.0923,1.0387
3,20240219,1.0145,0.9781,1.0800,0.9565,0.9852,1.0149,1.0145,0.9950,1.0359,...,1.0999,1.0534,1.1001,1.0138,1.0440,1.0000,1.0610,1.0639,1.0096,0.9962
4,20240220,1.0010,1.0061,1.0144,1.0455,1.0025,1.0490,0.9952,1.0000,1.0396,...,1.0306,1.0127,1.0618,1.0045,1.0289,1.0154,1.0176,1.0062,1.0047,1.0130
5,20240221,1.0998,1.0313,1.0218,1.0435,1.0100,1.0023,1.0096,0.9858,1.0429,...,1.0248,1.0233,0.9799,1.0226,1.0052,1.0249,1.0565,1.0010,1.0000,1.0098
6,20240222,1.0093,0.9941,1.0455,1.0556,0.9975,0.9837,1.0047,1.0102,1.0137,...,1.0502,1.0385,1.0865,1.0133,1.0189,1.0016,1.0193,1.0237,1.0220,1.0007
7,20240223,0.9954,1.0049,1.0586,1.0526,1.0099,0.9763,1.0142,0.9992,1.0450,...,1.0524,1.0589,1.0221,0.9956,1.0253,1.0271,1.0641,1.0241,1.0231,1.0149
8,20240226,0.9705,0.9784,1.0335,1.0500,0.9902,1.0267,1.0047,0.9941,1.0216,...,1.0315,1.0795,1.0000,0.9781,1.0107,1.0133,1.0014,1.0186,1.0120,1.0140
9,20240227,0.9972,1.0080,1.0998,1.0476,1.0248,0.9953,1.0093,1.0101,1.0127,...,1.0347,1.0103,1.0341,1.0022,1.0460,1.0200,1.0274,1.0308,1.0461,1.0355


In [13]:
def trans_frame_to_id(dataframe):
    train_data = dataframe.iloc[:, 1:]
    # 对所有行，都取前10个最大的
    def top_n(row, n):
        # return row.nlargest(n).values
        return row.nlargest(n).index.tolist()

    n = 20
    data_top_10 = train_data.apply(top_n, axis=1, n=n)

    # 将结果转换为 [266, 10] 的形状
    data_transformed = pd.DataFrame(data_top_10.tolist(), index=train_data.index)

    def to_id(row):
        return encode(row)
    
    data_transformed = data_transformed.apply(to_id, axis=1)
    data_transformed = torch.stack([torch.tensor(row) for row in data_transformed])
    return data_transformed

train_data = trans_frame_to_id(pd_train_data)
val_data = trans_frame_to_id(pd_val_data)

print(f'train.shape is {train_data.shape}')


train.shape is torch.Size([266, 20])


In [14]:
# 写个python程序，从pandas数据中获取一批训练数据：
# 1. 读取pandas数据A，格式是：[trade_date, label1, label2, label3，。。。]，date的样例有：20230104，label*是数字，样例有1.0399、0.9943
# 2. X是生成指定格式的shape = [shape, ]

import random

random.seed(42)
torch.manual_seed(33)

def print_data_info(split, index_data, index_id):
    data = pd_train_data if split == 'train' else pd_val_data
    data = pd_train_data

    print(f'{split}, code is {decode(index_id)}, date is {data.iloc[index_data, 0]},price_chg is {data.iloc[index_data, (index_id+1)*2]}, aount_chg is {data.iloc[index_data, (index_id+1)*2-1]}')


def get_batch(split):
    data = train_data if split == 'train' else val_data
    data = val_data

    indices = torch.randint(len(data)-1-block_size, (batch_size, ))

    # (batch, block)
    x = torch.stack([data[i:i+block_size] for i in indices])
    x = x.gather(2, torch.randint(x.shape[2], (x.shape[0], x.shape[1], 1))).squeeze(-1)

    # (batch, block)
    y = torch.stack([data[i+1:i+1+block_size] for i in indices])
    y = y.gather(2, torch.randint(y.shape[2], (y.shape[0], y.shape[1], 1))).squeeze(-1)

    # index = indices[0].item()
    # print(f'index is {index}, raw_data is {data[index:index+block_size+1]}')
    # print(f'get train_data x is {x[0]}, y is {y[0]}')
    # print(f'get train_data_src x is {[decode(i.item()) for i in x[0]]}, y is {y[0]}')

    return x, y

get_batch('val')



(tensor([[2184, 3509, 2807, 2366, 1955],
         [2495, 2599, 2717,  722, 2110],
         [1761, 1521, 3525, 2231, 1660],
         [3525, 1883, 1783, 2423,  782],
         [2411, 2523,  681, 2742, 2502],
         [3060, 2750, 2024,  789, 2633],
         [1840, 1955, 2097, 1566, 1728],
         [1983, 2694, 2554, 1850, 2638],
         [2083, 2197, 2747, 1631, 3026],
         [1613, 1702, 1781, 2764, 2654],
         [1603,  153,  141, 2418, 1698],
         [ 163, 2366,  730, 2262, 2234]]),
 tensor([[2573, 1477, 2366, 1700, 1632],
         [2245, 2207, 1901, 2231, 2184],
         [2526, 2207, 2200, 2797, 2789],
         [1783, 2184, 2632, 2071, 2046],
         [1946, 2011, 3952, 2633, 1115],
         [1811, 1675, 2658,  321, 2418],
         [1695, 2082, 1714, 2005, 2772],
         [2573, 1534, 2621, 1781, 2599],
         [1889, 1318, 2665, 1676,  723],
         [1850, 1761, 2245, 2717,  157],
         [ 535,  768, 2519, 1889, 2753],
         [1625, 1771, 1598, 2183, 1946]]))

In [15]:
# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9

# model init
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=meta_vocab_size, dropout=dropout) # start with model_args from command line
if init_from == 'scratch':
    # init a new model from scratch
    print("Initializing a new model from scratch")
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)


model.to(device)

# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

# optimizer
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)

checkpoint = None # free up memory



# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

# logging
if wandb_log and master_process:
    import wandb
    wandb.init(project=wandb_project, name=wandb_run_name, config=config)

X, Y = get_batch('train')

# 遍历并打印模型中的所有权重参数
# for name, param in model.named_parameters():
#     print(f"{name}: {param.shape}, {param}")

print(f'X is {X}')
print(f'Y is {Y}')
logits, loss = model(X, Y)
print(logits.shape)
# print(f'logits is {logits}, shape is {logits.shape}')
# print(f'loss is {loss}, loss.shape is {loss.shape}')


# print(model)


Initializing a new model from scratch
config is GPTConfig(block_size=5, vocab_size=4096, n_layer=8, n_head=8, n_embd=512, dropout=0, bias=False)
number of parameters: 27.27M
num decayed parameter tensors: 34, with 27,265,536 parameters
num non-decayed parameter tensors: 17, with 8,704 parameters
using fused AdamW: False
X is tensor([[3769, 2844, 2243, 2490, 2708],
        [3512,  851, 1575,  912, 1199],
        [1907, 1370, 1769, 2014, 1825],
        [2228,  933, 2011, 3378, 2794],
        [1969, 1844, 3870, 1164,   33],
        [2958, 1892, 3068,  479, 3092],
        [2573,  933, 1208, 1823, 2012],
        [1597, 2004, 1788, 2684,  211],
        [1024,  992,  424,  925,  213],
        [3031, 3297,  577, 2248, 2745],
        [ 706, 3160, 2246,  272,  129],
        [ 464, 2949, 1216,  778, 1286]])
Y is tensor([[ 179, 3741, 2490, 1286, 3020],
        [ 851,  690, 2229, 1502, 2400],
        [2997, 1800, 3330,  465, 1821],
        [1062, 2730, 1633, 3466, 2127],
        [2774,  265,  722, 

/home/ren/miniconda3/envs/nanogpt/lib/python3.12/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [16]:


# training loop
# X, Y = get_batch('train') # fetch the very first batch
t0 = time.time()
local_iter_num = 0 # number of iterations in the lifetime of this process
raw_model = model # unwrap DDP container if needed
running_mfu = -1.0
while True:
    # evaluate the loss on train/val sets and write checkpoints
    if iter_num % eval_interval == 0 and master_process:
        losses = estimate_loss()
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        
        if losses['val'] < best_val_loss or always_save_checkpoint:
            best_val_loss = losses['val']
            if iter_num > 0:
                checkpoint = {
                    'model': raw_model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'config': config,
                }
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))
                
    if iter_num == 0 and eval_only:
        break
    
    X, Y = get_batch('train')
    logits, loss = model(X, Y)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    # timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0 and master_process:
        # get loss as float. note: this is a CPU-GPU sync point
        # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
        lossf = loss.item() * gradient_accumulation_steps
        if local_iter_num >= 5: # let the training loop settle a bit
            mfu = raw_model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
            running_mfu = mfu if running_mfu == -1.0 else 0.9*running_mfu + 0.1*mfu
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms, mfu {running_mfu*100:.2f}%")
    iter_num += 1
    local_iter_num += 1

    # termination conditions
    if iter_num > max_iters:
        break



step 0: train loss 8.3249, val loss 8.3654
iter 0: loss 8.4224, time 280.24ms, mfu -100.00%
iter 1: loss 8.4369, time 152.26ms, mfu -100.00%
iter 2: loss 8.2479, time 149.11ms, mfu -100.00%
iter 3: loss 8.3589, time 150.68ms, mfu -100.00%
iter 4: loss 8.0871, time 147.20ms, mfu -100.00%
iter 5: loss 8.1896, time 145.44ms, mfu 0.02%
iter 6: loss 8.1562, time 128.98ms, mfu 0.02%
iter 7: loss 8.5244, time 132.94ms, mfu 0.02%
iter 8: loss 8.1771, time 150.04ms, mfu 0.02%
iter 9: loss 8.1805, time 153.81ms, mfu 0.02%
iter 10: loss 8.5046, time 158.50ms, mfu 0.02%
iter 11: loss 8.0154, time 150.91ms, mfu 0.02%
iter 12: loss 8.8403, time 152.58ms, mfu 0.02%
iter 13: loss 8.0768, time 161.37ms, mfu 0.02%
iter 14: loss 8.1246, time 157.39ms, mfu 0.02%
iter 15: loss 8.1934, time 154.82ms, mfu 0.02%
iter 16: loss 8.3130, time 154.00ms, mfu 0.02%
iter 17: loss 8.4961, time 155.85ms, mfu 0.02%
iter 18: loss 8.3684, time 155.46ms, mfu 0.02%
iter 19: loss 8.2341, time 153.21ms, mfu 0.02%
iter 20: los